In [ ]:
import urllib.parse
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import RDF, RDFS, FOAF, OWL, XSD, DC, DCTERMS 
import pandas as pd

In [ ]:
base_uri = "http://www.sanitasicilia.it/resource/"

g = Graph()

sso = Namespace("http://www.sanitasicilia.it/ontology/")

g.bind("sso", sso)

ssr = Namespace("http://www.sanitasicilia.it/resource/")
g.bind("ssr", ssr)


In [ ]:
comuni_df = pd.read_csv("../datasets/ready/comuni_sicilia.csv")

comuni_df.apply(lambda x : print(x[2]))

In [ ]:
import urllib as ul

ul.parse.quote("AAA AA''''")

In [49]:
def urify(ns, testo):
    testo=testo.replace(" ","_").replace("\'","")
    return ns+urllib.parse.quote(testo)

def addTriples(row):
    res = URIRef(urify(base_uri, row[0]))
    g.add([res, URIRef(RDF.type), URIRef(sso.Comune)])
    g.add([res, sso.hasName, Literal(row[0], datatype=XSD.string)])
    g.add([res, sso.hasTotalPopulation, Literal(row[1], datatype=XSD.int)])
    g.add([res, sso.hasProvince, Literal(row[2], datatype=XSD.string)])
    g.add([res, sso.hasProvinceAcr, Literal(row[3], datatype=XSD.string)])

comuni_df = pd.read_csv("../datasets/ready/comuni_sicilia.csv")

comuni_df.apply(lambda row : addTriples(row), axis=1)

g.serialize(destination='../datasets/graphs/comuni.ttl', 
            format='turtle')

<Graph identifier=N17e84d89de1a439f8fbcd2f5ce73cdca (<class 'rdflib.graph.Graph'>)>